In [3]:
pip install pascal-voc-writer

In [16]:
!pip install --upgrade albumentations

     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 38.2MB 83kB/s 
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [57]:
# ######################################################
# VISUALIZATION OF CREATED AUGMENTED IMAGES AND BOUNDING BOXES
# #######################################################

import random
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import numpy as np
import albumentations as A


def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)
    plt.show()


def plot_example(image,bboxes):
    fig = plt.figure(figsize=(15,15))
    if bboxes is not None:
      for bbox in bboxes:
        x_min = int(bbox[0])
        y_min = int(bbox[1])
        x_max = int(bbox[2])
        y_max = int(bbox[3])
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 5)
      plt.imshow(image)
      plt.show()

def plotExamples(images,all_bboxes):
  i = 0
  for bboxes in all_bboxes:
    plot_example(images[i],bboxes)
    i+=1


In [60]:
import albumentations as A
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
# from bbaug import policies
import cv2
import os
from pascal_voc_writer import Writer
from xml.dom import minidom
import xml.etree.ElementTree as ET
import imgaug as ia
import imgaug.augmenters as iaa
import math
import random
import copy
import glob
from PIL import Image
import numpy as np
random.seed(7)



def readImage(filename):
    # Reading Image using PIL
    im_path = '/content/drive/MyDrive/Augmentations/images/' + filename 
    os.chdir('/content/drive/MyDrive/Augmentations/images')
    image = Image.open(filename)
    image = np.array(image)
    return image
  
def getCoordinates(filename):
  os.chdir('/content/drive/MyDrive/Augmentations/annotations')
  my_tree = ET.parse(filename)
  my_root = my_tree.getroot()

  #  getting the coordinates
  allbb = []
  for obj in my_root.findall('object'):
    classid = str(obj.find('name').text)
    bndbox = obj.find('bndbox')
    xmin =0
    ymin =0
    xmax =0
    ymax =0
    xmin = int(bndbox.find('xmin').text)
    xmax = int(bndbox.find('xmax').text)
    ymin = int(bndbox.find('ymin').text)
    ymax = int(bndbox.find('ymax').text)
    b = [xmin, ymin, xmax, ymax, classid]
    allbb.append(b)
  # print(allbb)
  return allbb


#  modidy the XML with new coordinates and store in a new place
def modifyXML_andSave(filename,new_image_name, bboxes):
  os.chdir('/content/drive/MyDrive/Augmentations/annotations')
  my_tree = ET.parse(filename)
  my_root = my_tree.getroot()
  my_root.find('filename').text = new_image_name
  for bbox in bboxes:
    name = bbox[-1]
    # print(name)
    for obj in my_root.findall('object'):
      obj_name = obj.find('name')
      if (obj_name.text == name):
        # print("right object found")
        bndbox = obj.find('bndbox')
        bndbox.find('xmin').text = str(bbox[0])
        bndbox.find('xmax').text = str(bbox[1])
        bndbox.find('ymin').text = str(bbox[2])
        bndbox.find('ymax').text = str(bbox[3])
        # bndbox.find('xmin').text = str("{:.2f}".format(bbox[0]))
        # bndbox.find('xmax').text = str("{:.2f}".format(bbox[1]))
        # bndbox.find('ymin').text = str("{:.2f}".format(bbox[2]))
        # bndbox.find('ymax').text = str("{:.2f}".format(bbox[3]))
  new_xml = new_image_name.split('.')[0]+'.xml'
  os.chdir('/content/drive/MyDrive/Augmentations/new_annotations')
  my_tree.write(new_xml)


def save_aug_image(filename,image):
  print(image.dtype)
  im = Image.fromarray(image)
  os.chdir('/content/drive/MyDrive/Augmentations/new_images')
  im.save(filename)
  

def main_call():
  # defining the transforms
  #  TRANSFORMS
  transform = A.Compose([
                        #  A.Resize(width = 1920,height=1080),
                         A.HorizontalFlip(p=0.5),
                         A.RGBShift(r_shift_limit=20,g_shift_limit=20,b_shift_limit=20,p=0.5),
                         A.OneOf([
                                  A.Blur(blur_limit =3,p=0.4),
                                  A.ColorJitter(brightness=0.3,contrast=0.2,p=0.4),
                         ],p=0.4),
                         A.Rotate(limit = 10,p=0.4)
  ],bbox_params = A.BboxParams(format ='pascal_voc'))

  # #############

  images = os.listdir('/content/drive/MyDrive/Augmentations/images')
  annotations = os.listdir('/content/drive/MyDrive/Augmentations/annotations')
  for image in images:
    img = readImage(image)
    ann_path = image.split('.')[0]+'.xml'
    annotation = annotations[annotations.index(ann_path)]
    bbox = getCoordinates(annotation)
    print("aaa")
    print(bbox)
    print("bbb")
    images_list = []
    saved_bbox = []
    for i in range(20):
      augmentations = transform(image= img , bboxes = bbox)
      augmented_img = augmentations['image']
      images_list.append(augmented_img)
      saved_bbox.append(augmentations['bboxes'])
      new_filename = image.split('.')[0]+'_'+str(i)+'.jpg'
      save_aug_image(new_filename,augmented_img)
      modifyXML_andSave(annotation,new_filename,augmentations['bboxes'])
      # print(len(augmentations['bboxes']))
    # plotExamples(images_list,saved_bbox)
    # print(saved_bbox)





if __name__ == '__main__':
  # getCoordinates('/content/drive/MyDrive/Augmentations/annotations/hm_0.xml')
  main_call()

Streaming output truncated to the last 5000 lines.
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
aaa
[[287, 504, 362, 599, 'person'], [534, 400, 578, 490, 'person'], [284, 502, 314, 538, 'helmet'], [534, 399, 557, 421, 'helmet'], [289, 517, 344, 572, 'helmet'], [541, 415, 577, 463, 'jacket']]
bbb
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
aaa
[[493, 289, 529, 374, 'person'], [499, 306, 522, 328, 'no_jacket'], [497, 289, 513, 307, 'helmet'], [941, 336, 957, 358, 'helmet'], [924, 359, 945, 396, 'jacket'], [902, 341, 959, 433, 'person'], [567, 55, 642, 118, 'truck']]
bbb
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
aaa
[[542, 401, 597, 493, 'person']]
bbb
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
aaa
[[843, 277, 857, 290, 'helmet'], [835, 294, 861, 324, '